In [ ]:
import re #to split the text file
import random #to choose better
import requests #to send the cards to anki
from concurrent.futures import ThreadPoolExecutor #to speed up sending the cards to anki

# Read the content of the file
file_path = "test your understanding.txt" #TODO you might have to change this
with open(file_path, "r") as file:
    content = file.read()

# Split the content at "Test your understanding"
substrings = content.split("Test your understanding")

# Initialize a nested list to store non-empty lines after each substring
nested_list = []

#split before new task
pattern = re.compile(r'\n(?=.{1,2}\))')

# Process each substring and split non-empty lines
for substring in substrings[1:]:  # Start from index 1 to skip the first empty substring
    lines = re.split(pattern, substring)
    nested_list.append(lines[1:]) #first line is also empty

In [ ]:
# Draw one tut at random from the nested list
random_tut = random.choice(nested_list)
# Identify which element in the outer list it is from
index_of_random_element = nested_list.index(random_tut)
print(f"Tut: {index_of_random_element+1}")

# Print a random question form the tut
print(random.choice(random_tut))

In [3]:
#TODO make shure Anki is installed and currently running on the machine
#TODO make shure anki-connect is installed https://github.com/FooSoft/anki-connect
#TODO make shure you have created an deck with the appropriate deck name 

#put all in Anki list without Answer
anki_list = [{"note_type": "Basic",
               "fields": {"Front":f"Tut {tut_i+1}\n{q}",
                                                "Back":"Answer"},
                "tags": ["Tut " + str(tut_i+1)]} for tut_i, tut in enumerate(nested_list) for q_i, q in enumerate(tut)]

def create_anki_card(payload):
    anki_url = "http://localhost:8765"  # AnkiConnect default URL
    response = requests.post(anki_url, json=payload)
    return response.json()

def create_anki_cards(cards_data):
    deck_name = "ML 2023 Test your understanding"  #TODO Change this to your deck name
    tags = ["ML", "Data Science"]
    with ThreadPoolExecutor() as executor:
        payloads = []

        for card in cards_data:
            note_type = card["note_type"]
            fields = card["fields"]
            tags.extend(card["tags"])

            # Construct the payload for creating a new note
            payload = {
                "action": "addNote",
                "version": 6,
                "params": {
                    "note": {
                        "deckName": deck_name,
                        "modelName": note_type,
                        "fields": fields,
                        "tags": tags,  # Add tags if needed
                    }
                }
            }

            payloads.append(payload)

        # Use ThreadPoolExecutor to parallelize the requests
        responses = list(executor.map(create_anki_card, payloads))

        # Print the responses (optional)
        for response in responses:
            print(response)

create_anki_cards(anki_list) #Took half a minute on my machine